# Implementation of the redactle game

In [42]:
!pip install pandas

  Using cached pandas-2.0.0-cp39-cp39-win_amd64.whl (11.3 MB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)


In [41]:
import json
import nltk
import numpy as np
import pandas as pd
from transformers import pipeline
from utils import mask_sentence, propose_best_word, render_list_as_text, update

ModuleNotFoundError: No module named 'pandas'

## Initialization

In [2]:
# Load JSON data from file
with open('articles/imam.json', 'r') as f:
    data = json.load(f)

# Extract titre and article contents
titre = data['titre']
article = data['article']

In [3]:
titre_masked, titre_dico = mask_sentence(titre)

In [4]:
article_masked, article_dico = mask_sentence(article)

In [5]:
with open('included_words.txt', 'r') as file:
    # Read the lines from the file
    lines = file.readlines()
    
    # Remove any leading or trailing whitespaces from each line
    words = [line.strip().strip('"\',') for line in lines]
    
# Print the resulting list of words
print(words)

['L', 'de', '(', ')', 'en', '', 'est', 'une', 'le', 'la', 'à', 'pour', 'entre', 'des', 'd', 'qui', 'Ces', 'sur', 'les', 'n', ':', '«', 'plus', '»', 'du', 's', 'y', 'ainsi', '', 'Le', 'dans', 'le', 'l', 'par', 'est', 'de', 'comme', 'une', 'un', '<unk>', '.', 's', 'e', ' ', 'l', '-', 'et', '\\', '', 'es', 'é', '', 'nt', 'r', 'c', 'p', 'ts', 'ent', 'P', 'tr', 'ence', 'con', ').', 'D', 'WtXmlEndTag', '/', 'ra', 'm', 'in', 'esp', 'ssement', 'ation', 'nit', 'ard', '[', ']', 'ron', 'ité', '’', 'pl', '_', 'pr', 'mag', 'Dé', 'sse', 'di', 'it', 'gar', 'In', 'eur', 'ri', 'mi', 'ant', 'é', 'er', 'ur']


## Update

In [6]:
tried_words = words

In [7]:
llm_model  = pipeline("fill-mask", model="camembert/camembert-base-wikipedia-4gb", tokenizer="camembert/camembert-base-wikipedia-4gb", top_k = 20)

In [8]:
print(f"titre masked : {render_list_as_text(titre_masked)}")
print(f"article masked : {render_list_as_text(article_masked)}")

titre masked : <mask>
article masked : <mask> <mask> ( <mask> <mask> des <mask> <mask> <mask> qui est <mask> ) est une <mask> qui <mask> la <mask> en <mask> . <mask> <mask> est de <mask> une <mask> qui <mask> <mask> <mask> en <mask> qui <mask> les <mask> et la <mask> <mask> <mask> de l <mask> <mask> . <mask> les <mask> <mask> <mask> d <mask> une <mask> <mask> de l <mask> <mask> <mask> l <mask> <mask> est le <mask> <mask> et <mask> de la <mask> <mask> . <mask> les <mask> <mask> <mask> <mask> <mask> le <mask> de <mask> <mask> d <mask> <mask> et <mask> de <mask> <mask> <mask> <mask> d <mask> <mask> est plus <mask> dans le <mask> . <mask> les <mask> <mask> <mask> <mask> l <mask> <mask> est le <mask> <mask> . <mask> le <mask> du <mask> <mask> <mask> <mask> <mask> la <mask> d <mask> <mask> à <mask> du <mask> <mask> du <mask> <mask> . <mask> <mask> <mask> l <mask> <mask> <mask> <mask> <mask> <mask> d <mask> une <mask> <mask> : <mask> est <mask> par la <mask> <mask> et <mask> <mask> à <mask> <

In [9]:
temperature = 10

In [10]:
suggestion, tried_words = propose_best_word(render_list_as_text(article_masked), temperature, tried_words, llm_model)

In [11]:
titre_dico, titre_masked, article_dico, article_masked, good_answer = update(titre_dico, titre_masked, article_dico, article_masked, suggestion)

In [12]:
suggestion

'que'

In [13]:
print(f"titre masked : {render_list_as_text(titre_masked)}")
print(f"article masked : {render_list_as_text(article_masked)}")

titre masked : <mask>
article masked : <mask> <mask> ( <mask> <mask> des <mask> <mask> <mask> qui est <mask> ) est une <mask> qui <mask> la <mask> en <mask> . <mask> <mask> est de <mask> une <mask> qui <mask> <mask> <mask> en <mask> qui <mask> les <mask> et la <mask> <mask> <mask> de l <mask> <mask> . <mask> les <mask> <mask> <mask> d <mask> une <mask> <mask> de l <mask> <mask> <mask> l <mask> <mask> est le <mask> <mask> et <mask> de la <mask> <mask> . <mask> les <mask> <mask> <mask> <mask> <mask> le <mask> de <mask> <mask> d <mask> <mask> et <mask> de <mask> <mask> <mask> <mask> d <mask> <mask> est plus <mask> dans le <mask> . <mask> les <mask> <mask> <mask> <mask> l <mask> <mask> est le <mask> <mask> . <mask> le <mask> du <mask> <mask> <mask> <mask> <mask> la <mask> d <mask> <mask> à <mask> du <mask> <mask> du <mask> <mask> . <mask> <mask> <mask> l <mask> <mask> <mask> <mask> <mask> <mask> d <mask> une <mask> <mask> : <mask> est <mask> par la <mask> <mask> et <mask> <mask> à <mask> <

In [ ]:
for _ in range(100):
    
    # Listen to the suggested word
    suggestion, tried_words = propose_best_word(render_list_as_text(article_masked), temperature, tried_words, llm_model)
    print(suggestion)

    # Update article and title
    titre_dico, titre_masked, article_dico, article_masked, good_answer = update(titre_dico, titre_masked, article_dico, article_masked, suggestion)

    if titre == titre_masked:
        print("Gagné !")
        break

## Test lemmatization

In [15]:
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

In [16]:
lemmatizer = WordNetLemmatizer()

In [18]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...


True

In [25]:
word = "grandes"
normalized_word = lemmatizer.lemmatize(word, wn.VERB)
print("Mot original :", word)
print("Mot normalisé :", normalized_word)

Mot original : grandes
Mot normalisé : grandes


Conclusion : nul avec nltk

In [30]:
import spacy

In [33]:
nlp = spacy.load("fr_core_news_sm")

In [35]:
word = "ate"
lemmatized_word = nlp(word)[0].lemma_ # Lemmatisation
normalized_word = nlp(word)[0].norm_  # Normalisation
print("Mot original :", word)
print("Mot lemmatisé :", lemmatized_word)
print("Mot normalisé :", normalized_word)

Mot original : ate
Mot lemmatisé : at
Mot normalisé : ate


Conclusion : pas terrible

## Epsilon greedy

In [36]:
epsilon = 0.1

In [37]:
from transformers import CamembertTokenizer

In [38]:
tokenizer = CamembertTokenizer.from_pretrained("camembert/camembert-base-wikipedia-4gb")

In [39]:
vocab = tokenizer.get_vocab()

In [43]:
final_article = ['<mask>', '<mask>', ',', '<mask>', '<mask>', '<mask>', ',', 'est', 'un', '<mask>', ',', '<mask>', 'et', '<mask>', '<mask>', 'de', '<mask>', '<mask>', 'et', 'de', '<mask>', ',', 'né', 'le', '<mask>', '<mask>', '<mask>', 'à', '<mask>', '(', 'actuelle', 'Pologne', ',', 'à', 'l', "'", '<mask>', 'dans', 'l', "'", '<mask>', '<mask>', 'en', '<mask>', ')', 'et', 'mort', 'le', '27', 'mars', '<mask>', 'à', '<mask>', '<mask>', '(', '<mask>', ',', '<mask>', ')', '.', '<mask>', '<mask>', 'est', 'l', "'", 'une', 'des', 'figures', 'les', 'plus', 'importantes', 'du', '<mask>', '<mask>', 'du', '<mask>', '<mask>', ',', '<mask>', 'des', '<mask>', '<mask>', 'et', '<mask>', '.', '<mask>', 'de', '<mask>', '<mask>', '<mask>', '<mask>', 'dans', 'le', 
'<mask>', '<mask>', 'de', 'l', "'", '<mask>', '<mask>', '<mask>', ',', 'tout', 'comme', 'pour', '<mask>', '<mask>', 'et', '<mask>', '<mask>', '.', 'Il', 'a', '<mask>', '<mask>', '<mask>', '<mask>', '<mask>', 'été', '<mask>', '<mask>', '<mask>', '', 'des', 'plus', 'grands', '<mask>', '.', '<mask>', '<mask>', 'a', '<mask>', 'l', "'", '<mask>', '<mask>', '<mask>', '<mask>', 'en', '<mask>', ',', '<mask>', '<mask>', 'par', 'l', "'", '<mask>', '<mask>', '<mask>', 'une', 'fois', 'par', 'an', 'à', 'un', '<mask>', 'ou', '<mask>', '<mask>', 'une', '<mask>', '<mask>', '.', '<mask>', '<mask>', 'de', 'la', '<mask>', '<mask>', 'des', '<mask>', '<mask>', 'et', '<mask>', ',', '<mask>', '<mask>', 'a', 'su', '<mask>', 'un', 'style', '<mask>', 'et', '<mask>', '.', 'Il', 'a', '<mask>', 'des', '<mask>', '<mask>', 'dans', '<mask>', '<mask>', '<mask>', 'et', '<mask>', 'de', 's', "'", '<mask>', 'à', 'l', "'", '<mask>', '<mask>', 'ainsi', 'qu', "'", '<mask>', '<mask>', '<mask>', '.', 'Il', 'a', '<mask>', '<mask>', '<mask>', '<mask>', '<mask>', 'dans', 'des', '<mask>', ',', '<mask>', '<mask>', 'dans', 'des', '<mask>', '<mask>', 'ou', '<mask>', '.']

In [47]:
len(set(final_article) - set(words)) / 200

0.11